# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps.datasets
import json
from api_keys import g_key
from pprint import pprint
import time

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
map_path = "Resources/weatherpy.csv"

heatmap = pd.read_csv(map_path)
heatmap.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,upernavik,98,GL,1584227715,98,72.79,-56.15,-14.098,3.60157
1,port alfred,95,ZA,1584227561,87,-33.59,26.89,68.000,4.00423
2,hilo,90,US,1584227630,78,19.73,-155.09,77.000,6.93470
3,puerto leguizamo,100,CO,1584227813,91,-0.19,-74.78,74.048,1.72249
4,bengkulu,81,ID,1584227815,88,-3.80,102.27,75.182,3.08706


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [15]:
fig = gmaps.figure()
locations = heatmap[["Lat","Lng"]]
weights = heatmap["Humidity"]
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights = weights, point_radius = 10, max_intensity = 50))

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
hotel_df = heatmap.loc[(heatmap["Max Temp"] > 70) & (heatmap["Max Temp"] < 80) & (heatmap["Wind Speed"] < 10) & (heatmap["Cloudiness"] == 0), ["City", "Cloudiness", "Country", "Humidity", "Lat", "Lng", "Max Temp", "Wind Speed"]]
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location="
hotel_name = []

for index, row in hotel_df.iterrows():
    query_url = base_url + str(row["Lat"]) + "," + str(row["Lng"]) + "&radius=5000&type=lodging&key=" + g_key
    response = requests.get(query_url).json()
    time.sleep(0.5)
    try:
        hotel_name.append(response["results"][0]["name"])
    except:
        hotel_name.append("Not Found")
        pass
        
hotel_df["Hotel Name"] = hotel_name
hotel_df
    


,City,Cloudiness,Country,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
120,maxixe,0,MZ,82,-23.86,35.35,77.630,6.55441,Oceano
205,marawi,0,PH,82,8.00,124.29,75.992,2.99758,Derogongan Residence
371,taoudenni,0,ML,12,22.68,-3.98,70.628,2.93047,Not Found
469,ures,0,MX,36,29.43,-110.40,75.992,5.01088,Hotel HACIENDA San Francisco
506,xai-xai,0,MZ,96,-25.05,33.64,70.880,1.81197,Platinum Hotel
561,rey bouba,0,CM,14,8.67,14.18,70.394,1.61064,Not Found


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
filtered_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != "Not Found", ["Hotel Name", "City", "Country", "Lat", "Lng"]]

hotel_markers_list = []
hotel_markers_dict = {}
for index, row in filtered_hotel_df.iterrows():
    hotel_markers_dict["Hotel Name"] = row["Hotel Name"]
    hotel_markers_dict["location"] = (row["Lat"], row["Lng"])
    hotel_markers_dict["City"] = row["City"]
    hotel_markers_dict["Country"] = row["Country"]
    hotel_markers_list.append(hotel_markers_dict)
    hotel_markers_dict = {}


In [12]:
# NOTE: Do not change any of the code in this cell
city_locations = [city["location"] for city in hotel_markers_list]

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**city) for city in hotel_markers_list]



In [14]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(city_locations, info_box_content = hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig.add_layer(gmaps.heatmap_layer(locations, weights = weights, point_radius = 10, max_intensity = 200))

fig



# Display Map

Figure(layout=FigureLayout(height='420px'))